In [1]:
import pandas as pd
import numpy as np
# import datetime
from datetime import datetime, timezone
import requests
import json
import matplotlib.pyplot as plt
import math
import scipy.stats as st

In [2]:
def create_tuple(response):
    # all data
    response_data = response.json()
    # print(response_data)
    # data for a single target
    response_datum = response_data[0]
    # print(response_datum)
    # the target - e.g. "120MapleAve.STM.M22-V/AverageMassFlow"
    response_target = response_datum['target']
    # the metric for this target - e.g. "Steam"
    #response_metric = response_datum['metric']
    # print(response_metric)
    # the datapoints as a list of lists
    response_datapoints = response_datum['datapoints']
    # print(response_datapoints)
    # array of tuples for numpy
    tuple_array = [tuple(x) for x in response_datapoints]
    # make a numpy array
    npa = np.array(tuple_array, dtype=[
                   ('value', np.double), ('ts', 'datetime64[ms]')])
    # npa = npa.astype([('value', np.double), ('ts', datetime)])

    # print("here")
    # for index, i in enumerate(npa):
    #   j = pd.to_datetime(i[1]).replace(tzinfo=timezone.utc).astimezone(tz="America/New_York")
    #   npa[index][1] = j
    return npa

In [3]:
def current_date():
    return str(datetime.now())

In [4]:
"""
Returns an array of tuples where tuple[0] is the metric, and tuple[1] is the time 
of measurement

Target: a WebCTRL point name in the form of a string 
Example: "#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/zone/zone_temp/trend_log"

Server: a string representing the WebCTRL server our point exists in
Example: "biotech_main"

Start: the start time we wish to be querying data about (in string format)
Example: "2021-12-31T11:00Z"

End: The end time we wish to be querying data about
Example: "2022-12-31T11:00Z"
"""
def fume_query(target,server, start,end):
  url = "https://ypsu0n34jc.execute-api.us-east-1.amazonaws.com/dev/query"
  data = {
      "range": {
        "from": start,
        "to": end,
      },
      "targets": [
        {
          "payload": {
            "schema": server,
          },
          "target": target
        }
      ],

    }
  request = requests.post(url, json=data)
  print(request)
  # print(request.json())
  return create_tuple(request)

In [5]:
"""
Returns a time series of the ithaca air temperature over the respective 
"""

def outside_temp(start,end):
  # https://ypsu0n34jc.execute-api.us-east-1.amazonaws.com/dev/query doesn't work
  # https://portal.emcs.cornell.edu/api/datasources/proxy/5/query works
  # https://portal-api.emcs.cucloud.net/query works
  url = "https://portal-api.emcs.cucloud.net/query"
  target = "GameFarmRoadWeatherStation.TAVG_H_F"
  data = {
    "range": {
        "from": start,
        "to": end
    },
    "targets": [
        {
            "target": target,
        }
    ]
  }
  request = requests.post(url, json=data)
  print(request)
#   print(request.json())
  return create_tuple(request)

In [6]:
"""
A switch in the energy calculations that allows us to avoid issues between times of years where it is hotter/colder outside.
"""

def coldorhot(cfm,external,internal):
    if external<=internal:
        #sensible heating equation
        return 1.08 * cfm * (internal - external)
    if external>internal:
        #enthalpy of air
        return 0.24 * cfm /13.333 * 60 * (external - internal)

In [7]:
from datetime import datetime
from dateutil import parser
import matplotlib.dates as mdates

def query(target_list, server, start, end):
  cfm_master = fume_query(target_list[0],server,start,end)
  sash_master = fume_query(target_list[3],server,start,end)
  occ_master = fume_query(target_list[2],server,start,end)
  internal_temp_master = fume_query(target_list[1],server,start,end)
  #external_temp_master = outside_temp(start,end)
  external_temp_master = fume_query(target_list[4], server, start, end)
  print("\nExternal Temp Master ", external_temp_master)

  cfm_list = pd.Series(data=[i[0] for i in cfm_master], index=[i[1] for i in cfm_master])
  print("CFM List: ", cfm_list)
  cfm_list = cfm_list[~cfm_list.index.duplicated()]
  print("CFM List new: ", cfm_list)

  sash_list = pd.Series(data=[i[0] for i in sash_master], index=[i[1] for i in sash_master])
  print("\nSash List: ", sash_list)
  sash_list = sash_list[~sash_list.index.duplicated()]
  print("\nSash List new: ", sash_list)

  occ_list = pd.Series(data=[i[0] for i in occ_master], index=[i[1] for i in occ_master])
  print("\nOCC List: ", occ_list)
  occ_list = occ_list[~occ_list.index.duplicated()]
  print("\nOCC List new: ", occ_list)
  
  internal_temp_list = pd.Series(data=[i[0] for i in internal_temp_master], index=[i[1] for i in internal_temp_master])
  print("\nInternal Temp List: ", internal_temp_list)
  internal_temp_list = internal_temp_list[~internal_temp_list.index.duplicated()]
  print("\nInternal Temp List new: ", internal_temp_list)

  external_temp_list = pd.Series(data=[i[0] for i in external_temp_master], index=[i[1] for i in external_temp_master])
  print("\nExternal Temp List: ", external_temp_list)
  # external_temp_list = pd.read_csv("C:/Users/Dan/Documents/GitHub/fume-hood-dashboard/game_farm/hist.csv", index_col=0).squeeze()
  external_temp_list.index = external_temp_list.index.astype('datetime64[ns]')
  print("\nExternal Temp List: ", external_temp_list)

  # hourly data frame
  df = pd.concat([cfm_list, sash_list, occ_list, internal_temp_list, external_temp_list], axis=1)
  df.columns = ["cfm", "sash" ,"occ", "internal temp", "external temp"]
  display(df)

  df["BTUh"] = 1.08 * df["cfm"] * (df["internal temp"] - df["external temp"])

  display(df)

In [8]:
def new_query(target_list, server, start, end):
  #external_temp_master = outside_temp(start,end)
  cfm_master = fume_query(target_list[0],server,start,end)
  sash_master = fume_query(target_list[3],server,start,end)
  occ_master = fume_query(target_list[2],server,start,end)
  internal_temp_master = fume_query(target_list[1],server,start,end)
  external_temp_master = fume_query(target_list[4], server, start, end)
  print("\nExternal Temp Master ", external_temp_master)
  
  cfm_list = pd.Series(data=[i[0] for i in cfm_master], index=[i[1] for i in cfm_master])
  print("CFM List: ", cfm_list)
  cfm_list = cfm_list[~cfm_list.index.duplicated()]
  print("CFM List new: ", cfm_list)

  sash_list = pd.Series(data=[i[0] for i in sash_master], index=[i[1] for i in sash_master])
  print("\nSash List: ", sash_list)
  sash_list = sash_list[~sash_list.index.duplicated()]
  print("\nSash List new: ", sash_list)

  occ_list = pd.Series(data=[i[0] for i in occ_master], index=[i[1] for i in occ_master])
  print("\nOCC List: ", occ_list)
  occ_list = occ_list[~occ_list.index.duplicated()]
  print("\nOCC List new: ", occ_list)
  
  internal_temp_list = pd.Series(data=[i[0] for i in internal_temp_master], index=[i[1] for i in internal_temp_master])
  print("\nInternal Temp List: ", internal_temp_list)
  internal_temp_list = internal_temp_list[~internal_temp_list.index.duplicated()]
  print("\nInternal Temp List new: ", internal_temp_list)

  external_temp_list = pd.Series(data=[i[0] for i in external_temp_master], index=[i[1] for i in external_temp_master])
  print("\nExternal Temp List: ", external_temp_list)
  # external_temp_list = pd.read_csv("C:/Users/Dan/Documents/GitHub/fume-hood-dashboard/game_farm/hist.csv", index_col=0).squeeze()
  external_temp_list.index = external_temp_list.index.astype('datetime64[ns]')
  print("\nExternal Temp List new: ", external_temp_list)
  print("\nLength of External Temp List new: ", len(external_temp_list))

  # interpolating External Temperature values
  ext_temp_hr = external_temp_list.tolist()
  ext_temp = []
  i = 0
  while i < (len(ext_temp_hr)-1):
    initial = ext_temp_hr[i]
    difference = (ext_temp_hr[i+1] - initial)/3
    ext_temp_5 = []
    for x in range(3):
      ext_temp_5.append(initial + difference*x)
    print("\n 3 Ext Temp Values: ", ext_temp_5)
    ext_temp = ext_temp + ext_temp_5
    i += 1
  ext_temp = ext_temp[3:] + [ext_temp_hr[-1]]
  print("\nList of New Ext Temp Values: ", ext_temp)

  # Internal Temperatures span multiple days, External Temperatures span one day
  new_external_temp_list = pd.Series(data=ext_temp, index=[i[1] for i in internal_temp_master[:len(ext_temp)]])
  print("\nNew External Temp List: ", new_external_temp_list)

  new_df = pd.concat([cfm_list, sash_list, occ_list, internal_temp_list, new_external_temp_list], axis=1)
  new_df.columns = ["cfm", "sash" ,"occ", "internal temp", "external temp"]
  display(new_df)
  new_df = new_df.dropna()
  
  new_df['BTUh'] = new_df.apply(lambda new_df: coldorhot(new_df['cfm'], new_df['external temp'], new_df['internal temp']), axis=1)
  print("\nFinal Data Frame: ")
  display(new_df)
  
  # new_df["BTUh"] = 1.08 * new_df["cfm"] * (new_df["internal temp"] - new_df["external temp"])
  # print("\nFinal Data Frame With Lambda: ")
  # display(new_df)

In [9]:
target_list = ["#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hoodvalve_flow/trend_log",
               "#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/zone/zone_temp/trend_log",
               "#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/occ_trend",
               "#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hood_sash",
               "#biotech/ground_flr_mech/building_hydronic_heating_syatems/reheat_heat_exchanger/oat"]
server = "biotech_main"
startnov = str(datetime(2021, 11, 17))
endnov = str(datetime(2021, 11, 18))

startjun = str(datetime(2021, 6, 17))
endjun = str(datetime(2021, 6, 18))

#query(target_list,server,start,end)
new_query(target_list,server,startnov,endnov)

new_query(target_list,server,startjun,endjun)


# Weather data:
#   biotech_oa_conditions/temperature

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>

External Temp Master  [(35.2, '2021-11-17T05:00:00.000') (35.3, '2021-11-17T05:15:00.000')
 (35.5, '2021-11-17T05:30:00.000') (35.6, '2021-11-17T05:45:00.000')
 (35.3, '2021-11-17T06:00:00.000') (35.1, '2021-11-17T06:15:00.000')
 (35.3, '2021-11-17T06:30:00.000') (35.7, '2021-11-17T06:45:00.000')
 (35.8, '2021-11-17T07:00:00.000') (35.5, '2021-11-17T07:15:00.000')
 (35.1, '2021-11-17T07:30:00.000') (34.5, '2021-11-17T07:45:00.000')
 (34.5, '2021-11-17T08:00:00.000') (34.7, '2021-11-17T08:15:00.000')
 (35.6, '2021-11-17T08:30:00.000') (35.7, '2021-11-17T08:45:00.000')
 (35.5, '2021-11-17T09:00:00.000') (34.3, '2021-11-17T09:15:00.000')
 (33.8, '2021-11-17T09:30:00.000') (34.5, '2021-11-17T09:45:00.000')
 (35.7, '2021-11-17T10:00:00.000') (35.5, '2021-11-17T10:15:00.000')
 (36.1, '2021-11-17T10:30:00.000') (36. , '2021-11-17T10:45:00.000')
 (37.3, '2021-11-17T11:00:00.000') (38.1, '2021-11-17T11:15:00.0

,cfm,sash,occ,internal temp,external temp
2021-11-17 05:00:00,224.62,3.3,1.0,72.2,35.300000
2021-11-17 05:05:00,223.15,3.3,1.0,72.2,35.366667
2021-11-17 05:10:00,224.13,3.3,1.0,72.2,35.433333
2021-11-17 05:15:00,224.13,3.3,1.0,72.2,35.500000
2021-11-17 05:20:00,224.62,3.3,1.0,72.2,35.533333
...,...,...,...,...,...
2021-11-19 04:35:00,222.66,2.4,0.0,70.3,NaN
2021-11-19 04:40:00,224.13,2.4,0.0,70.3,NaN
2021-11-19 04:45:00,225.10,2.4,0.0,70.3,NaN
2021-11-19 04:50:00,223.64,2.4,0.0,70.3,NaN



Final Data Frame: 


,cfm,sash,occ,internal temp,external temp,BTUh
2021-11-17 05:00:00,224.62,3.3,1.0,72.2,35.300000,8951.55624
2021-11-17 05:05:00,223.15,3.3,1.0,72.2,35.366667,8876.90700
2021-11-17 05:10:00,224.13,3.3,1.0,72.2,35.433333,8899.75404
2021-11-17 05:15:00,224.13,3.3,1.0,72.2,35.500000,8883.61668
2021-11-17 05:20:00,224.62,3.3,1.0,72.2,35.533333,8894.95200
...,...,...,...,...,...,...
2021-11-19 04:10:00,223.15,2.4,0.0,70.4,43.033333,6595.42140
2021-11-19 04:15:00,222.66,2.4,0.0,70.4,43.000000,6588.95472
2021-11-19 04:20:00,227.06,2.4,0.0,70.4,42.900000,6743.68200
2021-11-19 04:25:00,224.62,2.4,0.0,70.3,42.800000,6671.21400


<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>

External Temp Master  [(55. , '2021-06-17T04:00:00.000') (54.7, '2021-06-17T04:15:00.000')
 (54.2, '2021-06-17T04:30:00.000') (53.9, '2021-06-17T04:45:00.000')
 (53.4, '2021-06-17T05:00:00.000') (53.1, '2021-06-17T05:15:00.000')
 (52.4, '2021-06-17T05:30:00.000') (52.2, '2021-06-17T05:45:00.000')
 (51.8, '2021-06-17T06:00:00.000') (51.4, '2021-06-17T06:15:00.000')
 (51.1, '2021-06-17T06:30:00.000') (51.1, '2021-06-17T06:45:00.000')
 (50.5, '2021-06-17T07:00:00.000') (50.3, '2021-06-17T07:15:00.000')
 (50.6, '2021-06-17T07:30:00.000') (50.3, '2021-06-17T07:45:00.000')
 (49.9, '2021-06-17T08:00:00.000') (50. , '2021-06-17T08:15:00.000')
 (49.5, '2021-06-17T08:30:00.000') (49.5, '2021-06-17T08:45:00.000')
 (48.8, '2021-06-17T09:00:00.000') (48.5, '2021-06-17T09:15:00.000')
 (48.8, '2021-06-17T09:30:00.000') (48.6, '2021-06-17T09:45:00.000')
 (48.4, '2021-06-17T10:00:00.000') (48.6, '2021-06-17T10:15:00.0

,cfm,sash,occ,internal temp,external temp
2021-06-17 04:00:00,223.64,1.6,0.0,70.8,54.700000
2021-06-17 04:05:00,227.06,1.6,0.0,70.8,54.533333
2021-06-17 04:10:00,226.57,1.6,0.0,70.7,54.366667
2021-06-17 04:15:00,223.64,1.6,0.0,70.7,54.200000
2021-06-17 04:20:00,225.59,1.6,0.0,70.6,54.100000
...,...,...,...,...,...
2021-06-19 03:35:00,227.55,1.6,0.0,70.7,NaN
2021-06-19 03:40:00,222.66,1.6,0.0,70.7,NaN
2021-06-19 03:45:00,223.64,1.6,0.0,70.7,NaN
2021-06-19 03:50:00,223.15,1.6,0.0,70.7,NaN



Final Data Frame: 


,cfm,sash,occ,internal temp,external temp,BTUh
2021-06-17 04:00:00,223.64,1.6,0.0,70.8,54.700000,3888.652320
2021-06-17 04:05:00,227.06,1.6,0.0,70.8,54.533333,3988.990080
2021-06-17 04:10:00,226.57,1.6,0.0,70.7,54.366667,3996.694800
2021-06-17 04:15:00,223.64,1.6,0.0,70.7,54.200000,3985.264800
2021-06-17 04:20:00,225.59,1.6,0.0,70.6,54.100000,4020.013800
...,...,...,...,...,...,...
2021-06-19 03:10:00,223.15,1.6,0.0,70.8,74.600000,915.830496
2021-06-19 03:15:00,227.55,1.6,0.0,70.8,74.600000,933.888547
2021-06-19 03:20:00,226.57,1.6,0.0,70.8,74.566667,921.709803
2021-06-19 03:25:00,225.10,1.6,0.0,70.8,74.533333,907.625891
